In [ ]:
# CCSL+ WSSL on 2 embeddings.

import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import os
import numpy as np
import pandas as pd

import glob
import random

from torch.autograd import Variable
from torch.autograd import Function
from torch import optim

##################################################################################
Nc = 10 # Number of language classes
n_epoch = 10 # Number of epochs
IP_dim = 39
e_dim = 64*2

look_back1 = 20
look_back2 = 40

def lstm_data(f):

    df = pd.read_csv(f,encoding='utf-16',usecols=list(range(0,IP_dim)))
    dt = df.astype(np.float32)
    X=np.array(dt)

    Xdata1=[]
    Xdata2=[]
    Ydata1 =[]

    mu = X.mean(axis=0)
    std = X.std(axis=0)
    np.place(std, std == 0, 1)
    X = (X - mu) / std
    f1 = os.path.splitext(f)[0]

    #print(f1)
    clas=f1[42:46]
    #print(clas)

    if   clas == 'cant':
        Y = 0
    elif clas == 'indo':
        Y = 1
    elif clas == 'japa':
        Y = 2
    elif clas == 'kaza':
        Y = 3
    elif clas == 'kore':
        Y = 4
    elif clas == 'mand':
        Y = 5
    elif clas == 'russ':
        Y = 6
    elif clas == 'tibe':
        Y = 7
    elif clas == 'uygh':
        Y = 8
    elif clas == 'viet':
        Y = 9

    Y1=np.array([Y])


    for i1 in range(0,len(X)-look_back1,1):    #High resolution low context
        a=X[i1:(i1+look_back1),:]
        Xdata1.append(a)
    Xdata1=np.array(Xdata1)

    for i2 in range(0,len(X)-look_back2,2):     #Low resolution long context
        b=X[i2:(i2+look_back2):3,:]
        Xdata2.append(b)
    Xdata2=np.array(Xdata2)


    Xdata1 = torch.from_numpy(Xdata1).float()
    Xdata2 = torch.from_numpy(Xdata2).float()
    Ydata1 = torch.from_numpy(Y1).long()

    return Xdata1,Xdata2,Ydata1,Y

#################################################################################### Modifying e1e2inter2aa
class LSTMNet(torch.nn.Module):
    def __init__(self):
        super(LSTMNet, self).__init__()
        self.lstm1 = nn.LSTM(IP_dim, 400,bidirectional=True)
        self.lstm2 = nn.LSTM(2*400, 128,bidirectional=True)
        self.lstm3 = nn.LSTM(2*128, 64,bidirectional=True)

        self.fc_ha=nn.Linear(e_dim,128)
        self.fc_1= nn.Linear(128,1)
        self.sftmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        ht = x[-1]
        ht = torch.unsqueeze(ht, 0)
        ha = torch.tanh(self.fc_ha(ht))
        alpha = self.fc_1(ha)
        al = self.sftmax(alpha) # Attention vector

        T = list(ht.shape)[1]  #T=time index
        batch_size = list(ht.shape)[0]
        dim = list(ht.shape)[2]
        c = torch.bmm(al.view(batch_size, 1, T),ht.view(batch_size,T,dim))
        #print('c size',c.size())
        e = torch.squeeze(c,0)
        return e

class WSSL_Net(nn.Module):
    def __init__(self, model1,model2):
        super(WSSL_Net, self).__init__()
        self.model1 = model1
        self.model2 = model2

        self.att1=nn.Linear(e_dim,100)
        self.att2= nn.Linear(100,1)
        self.bsftmax = torch.nn.Softmax(dim=1)

        self.lang_classifier= nn.Sequential()
        self.lang_classifier.add_module('fc1',nn.Linear(e_dim, Nc, bias=False))


    def forward(self, x1,x2):
        e1 = self.model1(x1)
        e2 = self.model2(x2)
        ht_e = torch.cat((e1,e2), dim=0)
        ht_e = torch.unsqueeze(ht_e, 0)
        ha_e = torch.tanh(self.att1(ht_e))
        alp = torch.tanh(self.att2(ha_e))
        al= self.bsftmax(alp)
        Tb = list(ht_e.shape)[1]
        batch_size = list(ht_e.shape)[0]
        D = list(ht_e.shape)[2]
        u_vec = torch.bmm(al.view(batch_size, 1, Tb),ht_e.view(batch_size,Tb,D))
        u_vec = torch.squeeze(u_vec,0)

        lang_output = self.lang_classifier(u_vec)      # Output layer

        return (u_vec,lang_output, e1, e2)
###############################################################################################

model1 = LSTMNet()
model2 = LSTMNet()

model1.cuda()
model2.cuda()

model = WSSL_Net(model1,model2)
model.cuda()
optimizer = optim.SGD(model.parameters(),lr = 0.01, momentum= 0.9)

loss_lang = torch.nn.CrossEntropyLoss(reduction='mean')
loss_ccsl = torch.nn.CosineSimilarity()
loss_wssl = torch.nn.CosineSimilarity()


loss_lang.cuda()
loss_ccsl.cuda()


manual_seed = random.randint(1,10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

files_list=[]
folders = glob.glob('/wd/users/t20287/tf1/project_m/mfcc_train/*')
for folder in folders:
    for f in glob.glob(folder+'/*.csv'):
        files_list.append(f)

l = len(files_list)
random.shuffle(files_list)
print('Total Training files: ',l)


e1_cntrd = {}
for o in range(Nc):  #Initialize centroids, it will be over-written after first epoch
    e1_cntrd[o] = np.zeros((1,e_dim))

'''e2_cntrd = {}
for o in range(Nc):  #Initialize centroids, it will be over-written after first epoch
    e2_cntrd[o] = np.zeros((1,e_dim))'''

e1_tot = {}
#e2_tot = {}
T_lang = {}
for o in range(Nc):
    e1_tot[o] = np.zeros((1,e_dim),dtype = np.float32) # Array for storing the total of all e1 embeddings (class-wise)
    #e2_tot[o] = np.zeros((1,e_dim),dtype = np.float32)
    T_lang[o] = 0  # Initialize total number of samples per language=0, will be incremented in the loop.


e1cntrd_gpu = {}
#e2cntrd_gpu = {}
err_e1 = np.zeros((1,Nc))
#err_e2 = np.zeros((1,Nc))
sftmx = torch.nn.Softmax(dim=1)

i = 0
cost = 0.
##############################################################################
for fn in files_list:
    #print(fn)
    df = pd.read_csv(fn,encoding='utf-16',usecols=list(range(0,IP_dim)))
    data = df.astype(np.float32)
    X = np.array(data)
    N,D=X.shape

    if N>look_back2:
        model.zero_grad()

        XX1,XX2,YY1,Yint = lstm_data(fn)
        XNP=np.array(XX1)
        if(np.isnan(np.sum(XNP))):
            continue

        XNP=np.array(XX2)
        if(np.isnan(np.sum(XNP))):
            continue

        i = i+1
        XX1 = np.swapaxes(XX1,0,1)
        XX2 = np.swapaxes(XX2,0,1)
        X1 = Variable(XX1,requires_grad=False).cuda()
        Y1 = Variable(YY1,requires_grad=False).cuda()
        X2 = Variable(XX2,requires_grad=False).cuda()


        uvec,fl, e1, e2 = model.forward(X1,X2)
        err_l = loss_lang(fl,Y1)

        T_err = err_l

        T_err.backward()
        optimizer.step()
        cost = cost + T_err.item()

        print("2ccsl1.py first batch: completed files  "+str(i)+"/"+str(l)+" Loss= %.3f"%(cost/i))

        if(i%100==0):
            break

################################################################################

print(" ########################################################################################################  ")

batch = 1  # Number of batches completed
for e in range(n_epoch):
    cost = 0.
    random.shuffle(files_list)
    i=0  # number of files completed in the epoch
    nf = 0 # Number of files completed/ for centroid computation
    batch_filelist =[]
    print("Extracting embeddings for batch: ",batch)

    for fn in files_list:
        batch_filelist.append(fn)  # List of file names in the mini batch
        df = pd.read_csv(fn,encoding='utf-16',usecols=list(range(0,IP_dim)))
        data = df.astype(np.float32)
        X = np.array(data)
        N,D=X.shape

        if N>look_back2:
            model.zero_grad()

            XX1,XX2,YY1,Yint = lstm_data(fn)
            XNP=np.array(XX1)
            if(np.isnan(np.sum(XNP))):
                continue

            XNP=np.array(XX2)
            if(np.isnan(np.sum(XNP))):
                continue

            nf =nf + 1
            XX1 = np.swapaxes(XX1,0,1)
            XX2 = np.swapaxes(XX2,0,1)
            X1 = Variable(XX1,requires_grad=False).cuda()
            Y1 = Variable(YY1,requires_grad=False).cuda()
            X2 = Variable(XX2,requires_grad=False).cuda()

            uvec,fl, e1, e2 = model.forward(X1,X2)
            e1c = uvec.cpu().detach().numpy() # Copy of e1 in cpu
            #e2c = e2.cpu().detach().numpy() # Copy of e2 in cpu

            e1_tot[Yint] = e1_tot[Yint] + e1c  # Add e1 according to language id
            #e2_tot[Yint] = e2_tot[Yint] + e2c  # Add e2 according to language id
            T_lang[Yint] = T_lang[Yint] + 1   # T,e2cotal u-vec in given language id (Yint is true language label)


            if(nf%150==0):
                for o in range(Nc):
                    e1_cntrd[o] = e1_tot[o]/T_lang[o]
                    e1cntrd_gpu[o] = torch.from_numpy(e1_cntrd[o]).type(torch.float32).to("cuda") # Send to GPU
                    #e2_cntrd[o] = e2_tot[o]/T_lang[o]
                    #e2cntrd_gpu[o] = torch.from_numpy(e2_cntrd[o]).type(torch.float32).to("cuda")

                print("#########  Completed computing the Centroids for this Minibatch. Now training ###################################")

                for fn in batch_filelist:
                    df = pd.read_csv(fn,encoding='utf-16',usecols=list(range(0,IP_dim)))
                    data = df.astype(np.float32)
                    X = np.array(data)
                    N,D=X.shape

                    if N>look_back2:
                        model.zero_grad()

                        XX1,XX2,YY1,Yint = lstm_data(fn)
                        XNP=np.array(XX1)
                        if(np.isnan(np.sum(XNP))):
                            continue

                        XNP=np.array(XX2)
                        if(np.isnan(np.sum(XNP))):
                            continue

                        i = i+1
                        XX1 = np.swapaxes(XX1,0,1)
                        XX2 = np.swapaxes(XX2,0,1)
                        X1 = Variable(XX1,requires_grad=False).cuda()
                        Y1 = Variable(YY1,requires_grad=False).cuda()
                        X2 = Variable(XX2,requires_grad=False).cuda()

                        uvec,fl, e1, e2 = model.forward(X1,X2)
                        err_l = loss_lang(fl,Y1)
                        err_wssl = abs(loss_wssl(e1,e2))

                        for o in range(Nc):
                            err_e1[0,o] = abs(loss_ccsl(uvec,e1cntrd_gpu[o])).cpu().detach().numpy()
                            #err_e2[0,o] = abs(loss_ccsl(e2,e2cntrd_gpu[o])).cpu().detach().numpy()  #Compute similarity with all embeddings


                        ccsl_e1 = torch.from_numpy(err_e1).type(torch.float32).to("cuda")
                        #print(ccsl_e1)
                        prob1 = sftmx(ccsl_e1)  ############################### Take softmax on the similarity values
                        print(prob1)
                        e1_loss = loss_lang(prob1,Y1)

                        '''
                        ccsl_e2 = torch.from_numpy(err_e2).type(torch.float32).to("cuda")
                        #print(ccsl_e2)
                        prob2 = sftmx(ccsl_e2)
                        print(prob2)
                        print(Y1)
                        e2_loss = loss_lang(prob2,Y1)'''

                        T_err = err_l + 0.2*err_wssl + 0.5*e1_loss #+ 0.75*e2_loss

                        T_err.backward()
                        optimizer.step()
                        cost = cost + T_err.item()

                        #erre1 = e1_loss.cpu().detach().numpy() # erre1 is for printing
                        #erre2 = e2_loss.cpu().detach().numpy() # erre2 is for printing
                        #print("e1_loss=",erre1," e2_loss=",erre2)

                        print("AP_OLR 2ccsl_wssl1.py:  epoch "+str(e+1)+" completed files  "+str(i)+"/"+str(l)+"                               Loss= %.3f"%(cost/i))

                print("Completed training files in batch ",batch," Clear the totals for next batch")
                for o in range(Nc):
                    e1_tot[o] = np.zeros((1,e_dim),dtype = np.float32)
                    #e2_tot[o] = np.zeros((1,e_dim),dtype = np.float32)
                    T_lang[o] = 0  # Initialize total number of samples per language=0, will be incremented in the loop.

                batch_filelist = [] # Clear batch_filelist
                batch = batch+1


    path = "/wd/users/t20287/tf1/project_m/ccsl_wssl1/"+str(e+1)+".pth"
    torch.save(model.state_dict(),path)




